### Test DuckDB using NETS datasets

In [2]:
import duckdb

# tes direct select from dataframe
#duckdb.sql('SELECT * FROM df')

# test direct select from parquet file
# duckdb.query('SELECT * FROM parquet_scan(\'./data/NETS2014_LatLongs.parquet\')')

# test direct select from txt (csv) file
#duckdb.query('SELECT * FROM read_csv(\'./data/NETS2014_LatLongs.txt\', delim=\'\\t\', header=true, columns={\'DunsNumber\': \'VARCHAR\', \'Latitude\': \'DOUBLE\', \'Longitude\': \'DOUBLE\', \'LevelCode\': \'VARCHAR\'} )')

#duckdb.query('SELECT * FROM read_csv_auto(\'./data/NETS2014_Sales.txt\')')

#### Create a duckdb database

NOTE: the performance of persistent database is worse than in-memory database. However, all tempory tables will be lost after the connection is closed when using in-memory db. So, I suggest to use persistent db for NETS data.



In [22]:
# empty parameters means creating in-memory database
# con = duckdb.connect()

# you can also create a persistent database file
con = duckdb.connect('./data/duckdb_test.db')


#### Install and load spatial extension on this database


In [23]:
con.install_extension('spatial')
con.load_extension('spatial')

In [26]:
# Drop table nets2014_latlongs if exists
con.execute('DROP TABLE IF EXISTS nets2014_latlongs')

#### Create a table from 1.72 GB LatLongs.txt (2.2 seconds in-memory vs 4.9 seconds persistent)


In [27]:
con.sql('CREATE TABLE nets2014_latlongs AS SELECT * FROM read_csv_auto(\'./data/NETS2014_LatLongs.txt\')')

#### Check the table `nets2014_latlongs`

In [25]:
con.sql('SELECT * FROM nets2014_latlongs')

┌────────────┬──────────┬───────────┬───────────┬──────────────────────────┐
│ DunsNumber │ Latitude │ Longitude │ LevelCode │           geom           │
│  varchar   │  double  │  double   │  varchar  │         geometry         │
├────────────┼──────────┼───────────┼───────────┼──────────────────────────┤
│ 001000009  │  42.2973 │   71.2004 │ D         │ POINT (71.2004 42.2973)  │
│ 001000012  │  21.3846 │  157.9372 │ Z         │ POINT (157.9372 21.3846) │
│ 001000020  │  21.3846 │  157.9372 │ Z         │ POINT (157.9372 21.3846) │
│ 001000038  │  36.0823 │   79.8976 │ D         │ POINT (79.8976 36.0823)  │
│ 001000046  │  21.3094 │  157.8633 │ D         │ POINT (157.8633 21.3094) │
│ 001000053  │  21.3364 │  157.9185 │ Z         │ POINT (157.9185 21.3364) │
│ 001000061  │  21.3064 │  157.8602 │ Z         │ POINT (157.8602 21.3064) │
│ 001000087  │  21.3116 │  157.8611 │ Z         │ POINT (157.8611 21.3116) │
│ 001000095  │   41.401 │   79.8735 │ D         │ POINT (79.8735 41.401)   │

### Add a geometry column using latitude and longitude

In [11]:
con.execute('ALTER TABLE nets2014_latlongs ADD COLUMN geom geometry')

#### Update the geom column use latitute and longitude columns (2.9 seconds in-memory / 16.1 second on disk)

In [12]:
con.execute('UPDATE nets2014_latlongs SET geom = ST_Point(Longitude, Latitude)')

#### Test spatial query using the geometry column

In [13]:
# query the table using ST_WITHIN function
con.sql('SELECT * FROM nets2014_latlongs WHERE ST_Within(geom, ST_GeomFromText(\'POLYGON ((70.4 37.7, 75.5 37.7, 75.5 40.8, 70.4 40.8, 70.4 37.7))\'))')

┌────────────┬──────────┬───────────┬───────────┬─────────────────────────┐
│ DunsNumber │ Latitude │ Longitude │ LevelCode │          geom           │
│  varchar   │  double  │  double   │  varchar  │        geometry         │
├────────────┼──────────┼───────────┼───────────┼─────────────────────────┤
│ 001000731  │  40.2207 │   74.8733 │ D         │ POINT (74.8733 40.2207) │
│ 001000798  │  40.2178 │   74.8326 │ Z         │ POINT (74.8326 40.2178) │
│ 001000822  │  39.9903 │   75.1344 │ D         │ POINT (75.1344 39.9903) │
│ 001000863  │  40.0411 │   75.1445 │ D         │ POINT (75.1445 40.0411) │
│ 001000921  │  39.9305 │   75.1628 │ D         │ POINT (75.1628 39.9305) │
│ 001001010  │  40.0068 │   75.1585 │ D         │ POINT (75.1585 40.0068) │
│ 001001135  │  39.9153 │   75.0625 │ Z         │ POINT (75.0625 39.9153) │
│ 001001515  │  40.7774 │   73.2267 │ D         │ POINT (73.2267 40.7774) │
│ 001001895  │  40.7526 │   74.1206 │ Z         │ POINT (74.1206 40.7526) │
│ 001002273 

Here we can't create table from raw txt file, since it will complain about "could not convert string to double", since there are many empty values in the raw txt files. So, we need to specify the column types for each column, and we can also ignore_errors=true, so that it will ignore the empty values.

In [14]:
# create an array using these values:
# Sales90 SalesC90        Sales91 SalesC91        Sales92 SalesC92        Sales93 SalesC93        Sales94 SalesC94   Sales95 SalesC95        Sales96 SalesC96        Sales97 SalesC97        Sales98 SalesC98        Sales99 SalesC99  Sales00  SalesC00        Sales01 SalesC01        Sales02 SalesC02        Sales03 SalesC03        Sales04 SalesC04        Sales05    SalesC05        Sales06 SalesC06        Sales07 SalesC07        Sales08 SalesC08        Sales09 SalesC09        Sales10    SalesC10        Sales11 SalesC11        Sales12 SalesC12        Sales13 SalesC13        Sales14 SalesC14        SalesHere  SalesHereC      SalesGrowth     SalesGrowthPeer
sales_columns = ['Sales90', 'SalesC90', 'Sales91', 'SalesC91', 'Sales92',
       'SalesC92', 'Sales93', 'SalesC93', 'Sales94', 'SalesC94', 'Sales95',
       'SalesC95', 'Sales96', 'SalesC96', 'Sales97', 'SalesC97', 'Sales98',
       'SalesC98', 'Sales99', 'SalesC99', 'Sales00', 'SalesC00', 'Sales01',
       'SalesC01', 'Sales02', 'SalesC02', 'Sales03', 'SalesC03', 'Sales04',
       'SalesC04', 'Sales05', 'SalesC05', 'Sales06', 'SalesC06', 'Sales07',
       'SalesC07', 'Sales08', 'SalesC08', 'Sales09', 'SalesC09', 'Sales10',
       'SalesC10', 'Sales11', 'SalesC11', 'Sales12', 'SalesC12', 'Sales13',
       'SalesC13', 'Sales14', 'SalesC14', 'SalesHere', 'SalesHereC',
       'SalesGrowth', 'SalesGrowthPeer']
columns_sql = ', '.join(["'{}':'DOUBLE'".format(x) for x in sales_columns])


#### Create a table from 7.32 GB Sales.txt (37 seconds in-memory vs 1m 0.9s on disk)

In [15]:
con.sql("CREATE TABLE nets2014_sales AS SELECT * FROM read_csv('./data/NETS2014_Sales.txt', delim='\\t', header=true, ignore_errors=true, columns={{'DunsNumber': 'VARCHAR', {}}})".format(columns_sql))

#### Check the table `nets2014_sales`

In [16]:
con.sql('SELECT * FROM nets2014_sales')

┌────────────┬──────────┬──────────┬──────────┬───┬──────────┬───────────┬────────────┬─────────────┬─────────────────┐
│ DunsNumber │ Sales90  │ SalesC90 │ Sales91  │ … │ SalesC14 │ SalesHere │ SalesHereC │ SalesGrowth │ SalesGrowthPeer │
│  varchar   │  double  │  double  │  double  │   │  double  │  double   │   double   │   double    │     double      │
├────────────┼──────────┼──────────┼──────────┼───┼──────────┼───────────┼────────────┼─────────────┼─────────────────┤
│ 001000009  │ 450000.0 │      0.0 │  46000.0 │ … │     NULL │  112700.0 │        3.0 │         1.0 │             1.0 │
│ 001000012  │     NULL │     NULL │     NULL │ … │     NULL │  450000.0 │        3.0 │         2.0 │             2.0 │
│ 001000020  │     NULL │     NULL │     NULL │ … │     NULL │   80000.0 │        3.0 │         4.0 │             4.0 │
│ 001000038  │     NULL │     NULL │     NULL │ … │     NULL │  125000.0 │        0.0 │         2.0 │             2.0 │
│ 001000046  │     NULL │     NULL │    

#### Join the two tables using `DunsNumber` 

In [17]:
# create a new table by joining the two table using DunsNumber as key (20s in-memory vs 10s persistent)
con.sql('SELECT * FROM nets2014_latlongs JOIN nets2014_sales ON nets2014_latlongs.DunsNumber = nets2014_sales.DunsNumber')

┌────────────┬──────────┬───────────┬───────────┬───┬────────────┬────────────┬─────────────┬─────────────────┐
│ DunsNumber │ Latitude │ Longitude │ LevelCode │ … │ SalesHere  │ SalesHereC │ SalesGrowth │ SalesGrowthPeer │
│  varchar   │  double  │  double   │  varchar  │   │   double   │   double   │   double    │     double      │
├────────────┼──────────┼───────────┼───────────┼───┼────────────┼────────────┼─────────────┼─────────────────┤
│ 003581829  │  35.1684 │    89.791 │ D         │ … │   257100.0 │        3.0 │         4.0 │             4.0 │
│ 003581837  │   40.975 │   90.3602 │ D         │ … │  4682900.0 │        3.0 │         4.0 │             4.0 │
│ 003581838  │   28.583 │   81.2944 │ D         │ … │    50000.0 │        3.0 │         2.0 │             2.0 │
│ 003581839  │  45.4491 │   91.7859 │ D         │ … │   186000.0 │        3.0 │         2.0 │             2.0 │
│ 003581840  │  33.6843 │  117.8853 │ D         │ … │    81300.0 │        3.0 │         2.0 │           

In [19]:
# Create a index on the DunsNumber column
#con.execute('DROP INDEX idx_dunsnumber');
con.sql('CREATE UNIQUE INDEX idx_dunsnumber ON nets2014_latlongs(DunsNumber)')
#con.execute('DROP INDEX idx_dunsnumber_sales');
con.sql('CREATE UNIQUE INDEX idx_dunsnumber_sales ON nets2014_sales(DunsNumber)')

In [20]:
# Try join again
con.sql('SELECT * FROM nets2014_latlongs JOIN nets2014_sales ON nets2014_latlongs.DunsNumber = nets2014_sales.DunsNumber')

┌────────────┬──────────┬───────────┬───────────┬───┬────────────┬────────────┬─────────────┬─────────────────┐
│ DunsNumber │ Latitude │ Longitude │ LevelCode │ … │ SalesHere  │ SalesHereC │ SalesGrowth │ SalesGrowthPeer │
│  varchar   │  double  │  double   │  varchar  │   │   double   │   double   │   double    │     double      │
├────────────┼──────────┼───────────┼───────────┼───┼────────────┼────────────┼─────────────┼─────────────────┤
│ 001000009  │  42.2973 │   71.2004 │ D         │ … │   112700.0 │        3.0 │         1.0 │             1.0 │
│ 001000012  │  21.3846 │  157.9372 │ Z         │ … │   450000.0 │        3.0 │         2.0 │             2.0 │
│ 001000020  │  21.3846 │  157.9372 │ Z         │ … │    80000.0 │        3.0 │         4.0 │             4.0 │
│ 001000038  │  36.0823 │   79.8976 │ D         │ … │   125000.0 │        0.0 │         2.0 │             2.0 │
│ 001000046  │  21.3094 │  157.8633 │ D         │ … │    86700.0 │        3.0 │         2.0 │           

NOTE: the join is very slow, and adding index seems not improving. Since it will create a temporary table, and the temporary table is very large. So it's better to create a new table from the join result.

In [21]:
con.sql('CREATE TABLE latlon_sale AS SELECT * FROM nets2014_latlongs JOIN nets2014_sales ON nets2014_latlongs.DunsNumber = nets2014_sales.DunsNumber')

In [64]:
# reuse the joined table
con.sql('SELECT * FROM latlon_sales')

┌────────────┬──────────┬───────────┬───────────┬───┬────────────┬────────────┬─────────────┬─────────────────┐
│ DunsNumber │ Latitude │ Longitude │ LevelCode │ … │ SalesHere  │ SalesHereC │ SalesGrowth │ SalesGrowthPeer │
│  varchar   │  double  │  double   │  varchar  │   │   double   │   double   │   double    │     double      │
├────────────┼──────────┼───────────┼───────────┼───┼────────────┼────────────┼─────────────┼─────────────────┤
│ 004340027  │   35.923 │   80.2098 │ D         │ … │    70000.0 │        3.0 │         2.0 │             2.0 │
│ 004340031  │  36.0744 │  115.1947 │ D         │ … │   105000.0 │        3.0 │         2.0 │             2.0 │
│ 004340033  │  33.5296 │  112.2497 │ Z         │ … │    65000.0 │        3.0 │         2.0 │             2.0 │
│ 004340036  │  41.8606 │    88.297 │ D         │ … │    66700.0 │        3.0 │         2.0 │             2.0 │
│ 004340037  │  34.1223 │    80.873 │ D         │ … │   100000.0 │        3.0 │         2.0 │           

In [63]:
# list all tables
con.sql('SELECT * FROM sqlite_master')

┌─────────┬──────────────────────┬───────────────────┬──────────┬──────────────────────────────────────────────────────┐
│  type   │         name         │     tbl_name      │ rootpage │                         sql                          │
│ varchar │       varchar        │      varchar      │  int32   │                       varchar                        │
├─────────┼──────────────────────┼───────────────────┼──────────┼──────────────────────────────────────────────────────┤
│ table   │ latlon_sales         │ latlon_sales      │        0 │ CREATE TABLE latlon_sales(DunsNumber VARCHAR, Lati…  │
│ table   │ nets2014_latlongs    │ nets2014_latlongs │        0 │ CREATE TABLE nets2014_latlongs(DunsNumber VARCHAR,…  │
│ table   │ nets2014_sales       │ nets2014_sales    │        0 │ CREATE TABLE nets2014_sales(DunsNumber VARCHAR, Sa…  │
│ index   │ idx_dunsnumber_sales │ nets2014_sales    │        0 │ CREATE UNIQUE INDEX idx_dunsnumber_sales ON nets20…  │
│ index   │ idx_dunsnumber      